In [1]:
import numpy as np
from sklearn import svm

In [2]:
training_features = np.loadtxt("features.train")
testing_features = np.loadtxt("features.test")

In [3]:
def split_data(one_versus_all, dataset, choice1, choice2):
    to_return = dataset
    if not one_versus_all:
        to_return = []
        
        for digit, intensity, symmetry in dataset:
            if digit == choice1 or digit == choice2:
                to_return.append([digit, intensity, symmetry])

    to_return = np.array(to_return)
    digits = to_return[:, 0]
    intensity = to_return[:, 1]
    symmetry = to_return[:, 2]
    
    return digits, intensity, symmetry

In [4]:
def build_my_kernel(Q):
    def my_kernel(xn, xm):
        return (1 + np.dot(xn, xm.T)) ** Q
    
    return my_kernel

In [5]:
def generate_one_versus_all_yn(digits, one_choice):
    yn = np.ones(len(digits))
    
    for i, digit in enumerate(digits):
        if digit != one_choice:
            yn[i] = -1
            
    return np.array(yn)

In [6]:
def parse_digits(digits, choice1, choice2):
    result = []
    
    for digit in digits:
        if digit == choice1 or digit == choice2:
            result.append(digit)
            
    return digits

In [7]:
def generate_one_versus_one_yn(digits, choice1, choice2):
    parsed_digits = parse_digits(digits, choice1, choice2)
    yn = np.ones(len(parsed_digits))
    
    for i, digit in enumerate(digits):
        if digit == choice2:
            yn[i] = -1
    
    return yn

In [8]:
def calc_error(yn, y_pred):
    # misclassified points are opposite sign so will be -1 when multiplied
    error = yn * y_pred
    return np.count_nonzero(error == -1) / len(yn)

In [9]:
def run_svm(one_versus_all, choice, choice1=-1, C=0.01, Q=2):
    digits_train, intensities_train, symmetries_train = split_data(one_versus_all, training_features, choice, choice1)
    digits_test, intensities_test, symmetries_test = split_data(one_versus_all, testing_features, choice, choice1)
    
    if (one_versus_all):
        yn_train = generate_one_versus_all_yn(digits_train, choice)
        yn_test = generate_one_versus_all_yn(digits_test, choice)
    
    else:
        yn_train = generate_one_versus_one_yn(digits_train, choice, choice1)
        yn_test = generate_one_versus_one_yn(digits_test, choice, choice1)

    xn_train = np.column_stack((intensities_train, symmetries_train))
    xn_test= np.column_stack((intensities_test, symmetries_test))
    model = svm.SVC(C=C, kernel=build_my_kernel(Q), degree=Q, gamma=1)
    model.fit(xn_train, yn_train)
    
    y_train_pred = model.predict(xn_train)
    y_test_pred = model.predict(xn_test)
    
    Ein = calc_error(yn_train, np.array(y_train_pred))
    Eout = calc_error(yn_test, np.array(y_test_pred))
    
    return Ein, np.sum(model.n_support_), Eout

Q2

In [11]:
choices = [0, 2, 4, 6, 8]

for choice in choices:
    Ein, _, _ = run_svm(True, choice)
    print(f"E_in for {choice} vs. all: {Ein}")

E_in for 0 vs. all: 0.10588396653408312
E_in for 2 vs. all: 0.10026059525442327
E_in for 4 vs. all: 0.08942531888629818
E_in for 6 vs. all: 0.09107118365107666
E_in for 8 vs. all: 0.07433822520916199


Q3

In [12]:
choices = [1, 3, 5, 7, 9]

for choice in choices:
    Ein, _, _ = run_svm(True, choice)
    print(f"E_in for {choice} vs. all: {Ein}")

E_in for 1 vs. all: 0.014401316691811822
E_in for 3 vs. all: 0.09024825126868742
E_in for 5 vs. all: 0.07625840076807022
E_in for 7 vs. all: 0.08846523110684405
E_in for 9 vs. all: 0.08832807570977919


Q4

In [13]:
choice_from_q2 = 0

_, n_svs, _ = run_svm(True, choice_from_q2)
print(f"# support vectors for {choice} vs. all: {n_svs}")

# support vectors for 9 vs. all: 2179


In [14]:
choice_from_q3 = 1

_, n_svs, _ = run_svm(True, choice_from_q3)
print(f"# support vectors for {choice} vs. all: {n_svs}")

# support vectors for 9 vs. all: 386


In [22]:
2179 - 386

1793

Q5

In [15]:
# investigate a and b
C = [0.001, 0.01, 0.1, 1]

for c in C:
    _, n_svs, _ = run_svm(False, 1, 5, c)
    print(f"# support vectors for C={c}: {n_svs}")

# support vectors for C=0.001: 76
# support vectors for C=0.01: 34
# support vectors for C=0.1: 24
# support vectors for C=1: 24


In [16]:
# investigate c
for c in C:
    _, _, E_out = run_svm(False, 1, 5, c)
    print(f"E_out C={c}: {E_out}")

E_out C=0.001: 0.01650943396226415
E_out C=0.01: 0.018867924528301886
E_out C=0.1: 0.018867924528301886
E_out C=1: 0.018867924528301886


In [17]:
# investigate d
for c in C:
    E_in, _, _ = run_svm(False, 1, 5, c)
    print(f"E_in C={c}: {E_in}")

E_in C=0.001: 0.004484304932735426
E_in C=0.01: 0.004484304932735426
E_in C=0.1: 0.004484304932735426
E_in C=1: 0.0032030749519538757


Q6

In [18]:
# investigate a
E_in, _, _ = run_svm(False, 1, 5, 0.0001)
print(f"E_in for C=0.0001 and Q=2: {E_in}")
E_in, _, _ = run_svm(False, 1, 5, 0.0001, 5)
print(f"E_in for C=0.0001 and Q=5: {E_in}")

E_in for C=0.0001 and Q=2: 0.008968609865470852
E_in for C=0.0001 and Q=5: 0.004484304932735426


In [19]:
# investigate b
_, n_svm, _ = run_svm(False, 1, 5, 0.001)
print(f"# support vectors for C=0.001 and Q=2: {n_svm}")
_, n_svm, _ = run_svm(False, 1, 5, 0.001, 5)
print(f"# support vectors for C=0.001 and Q=5: {n_svm}")

# support vectors for C=0.001 and Q=2: 76
# support vectors for C=0.001 and Q=5: 25


In [20]:
# investigate c
E_in, _, _ = run_svm(False, 1, 5, 0.01)
print(f"E_in for C=0.01 and Q=2: {E_in}")
E_in, _, _ = run_svm(False, 1, 5, 0.01, 5)
print(f"E_in for C=0.01 and Q=5: {E_in}")

E_in for C=0.01 and Q=2: 0.004484304932735426
E_in for C=0.01 and Q=5: 0.003843689942344651


In [23]:
# investigate d
_, _, E_out = run_svm(False, 1, 5, 1)
print(f"E_out for C=1 and Q=2: {E_out}")
_, _, E_out = run_svm(False, 1, 5, 1, 5)
print(f"E_out for C=1 and Q=5: {E_out}")

E_out for C=1 and Q=2: 0.018867924528301886
E_out for C=1 and Q=5: 0.02122641509433962
